# 5.10 프로젝트 : 뉴스기사 분야별로 크롤링하여 저장!

In [ ]:
from urllib import parse

header = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}
# Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36
def getDownload(url, params={}, retries=3):
    resp = None
    
    try:
        resp = requests.get(url, params=params, headers=header)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and retries > 0:
            print(retries)
            resp = getDownload(url, params, retries-1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
            print(e.response.headers)
            
    return resp

# 크롤 관련 라이브러리들 import

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time

# 다음
## 사회, 정치, 경제, 국제, 문화, IT로 나눠서

In [ ]:
def daum_news_detail_crawl(links):
    for link in links:
        # 뉴스로만 향하는 링크들만
        if link.startswith("http://v.media.daum.net/v/"):

            article_id = link.split("/")[-1]

            html = getDownload(link)
            detail_dom = BeautifulSoup(html.content, "lxml")

            # 파일 생성
            file_name = "./news_crawl_project/" + field + "-" + article_id
            f = open(file_name, 'w', encoding="utf-8")

            for _ in detail_dom.select("#harmonyContainer > section > p"):
    #           각 p태그 파일에 write
                f.write(_.text)

            print(file_name, "생성 완료")
            time.sleep(1)
            f.close()

In [ ]:
browser = webdriver.Chrome()
url = "https://media.daum.net/"
browser.get(url)

### 사회

In [ ]:
# 분야이름 
field = browser.find_elements_by_css_selector("#kakaoGnb > div > ul > li:nth-child(2) > a")[0].text
# 사회분야 클릭
browser.find_elements_by_css_selector("#kakaoGnb > div > ul > li:nth-child(2) > a")[0].click()

### 페이지 소스 받아오기

In [ ]:
dom = BeautifulSoup(browser.page_source, "lxml")

### 세부링크 가져와서 urljoin -> url방문 -> 내용 긁어오기
#### 처음엔 생으로 짜고 이것을 함수로 만들기

In [ ]:
links = [_["href"] for _ in dom.select(".section_cate.section_headline .link_txt") if _.has_attr('href')]

for link in links:
    # 뉴스로만 향하는 링크들만
    if link.startswith("http://v.media.daum.net/v/"):

        article_id = link.split("/")[-1]

        html = getDownload(link)
        detail_dom = BeautifulSoup(html.content, "lxml")
        
        # 파일 생성
        file_name = "./news_crawl_project/" + field + "-" + article_id
        f = open(file_name, 'w', encoding="utf-8")
        
        for _ in detail_dom.select("#harmonyContainer > section > p"):
#           각 p태그 파일에 write
            f.write(_.text)
        
        print(file_name, "생성 완료")
        
        f.close()
        #harmonyContainer > section > p:nth-child(3)
        

## 정치

In [ ]:
field = browser.find_elements_by_xpath("//*[@id='kakaoGnb']/div/ul/li[3]/a")[0].text
browser.find_elements_by_xpath("//*[@id='kakaoGnb']/div/ul/li[3]/a")[0].click()

In [ ]:
dom = BeautifulSoup(browser.page_source, "lxml")
links = [_["href"] for _ in dom.select(".section_cate.section_headline .link_txt") if _.has_attr('href')]

daum_news_detail_crawl(links)

## 경제

In [ ]:
field = browser.find_elements_by_css_selector("#kakaoGnb > div > ul > li:nth-child(4) > a")[0].text
browser.find_elements_by_css_selector("#kakaoGnb > div > ul > li:nth-child(4) > a")[0].click()

dom = BeautifulSoup(browser.page_source, "lxml")

links = [_["href"] for _ in dom.select(".section_cate.section_headline .link_txt") if _.has_attr('href')]

daum_news_detail_crawl(links)

## 국제

In [ ]:
field = browser.find_elements_by_css_selector("#kakaoGnb > div > ul > li:nth-child(5) > a")[0].text
browser.find_elements_by_css_selector("#kakaoGnb > div > ul > li:nth-child(5) > a")[0].click()

dom = BeautifulSoup(browser.page_source, "lxml")

links = [_["href"] for _ in dom.select(".section_cate.section_headline .link_txt") if _.has_attr('href')]

daum_news_detail_crawl(links)

## 문화

In [ ]:
field = browser.find_elements_by_css_selector("#kakaoGnb > div > ul > li:nth-child(6) > a")[0].text
browser.find_elements_by_css_selector("#kakaoGnb > div > ul > li:nth-child(6) > a")[0].click()

dom = BeautifulSoup(browser.page_source, "lxml")

links = [_["href"] for _ in dom.select(".section_cate.section_headline .link_txt") if _.has_attr('href')]

daum_news_detail_crawl(links)

        #harmonyContainer > section > p:nth-child(3)
        

## IT

In [ ]:
field = browser.find_elements_by_css_selector("#kakaoGnb > div > ul > li:nth-child(7) > a")[0].text
browser.find_elements_by_css_selector("#kakaoGnb > div > ul > li:nth-child(7) > a")[0].click()

dom = BeautifulSoup(browser.page_source, "lxml")

daum_news_detail_crawl(links)

# 네이트 뉴스

In [ ]:
browser = webdriver.Chrome()
url = "https://news.nate.com"
browser.get(url)

In [ ]:
def nate_news_detail_crawl(links):
    print(field, "분야 개수 : ", len(links))
    for link in links:
        if link.startswith("//news.nate.com/view/"):
            article_id = link.split("/")[-1].split('?')[0]

            html = getDownload("https:" + link)
            detail_dom = BeautifulSoup(html.content, "lxml")

            # 파일 생성
            file_name = "./news_crawl_project/" + field + "-" + article_id
            f = open(file_name, 'w', encoding="utf-8")

            for _ in detail_dom.select("#realArtcContents"):
    #           각 p태그 파일에 write
                f.write(_.text)

            print(file_name, "생성 완료")
            time.sleep(1)

### 사회

In [ ]:
field = browser.find_element_by_class_name("news03").text
browser.find_element_by_class_name("news03").click()

In [ ]:
dom = BeautifulSoup(browser.page_source, "lxml")

links = [_["href"] for _ in dom.select("#newsContents a") if _.has_attr('href')]

nate_news_detail_crawl(links)

### 경제

In [ ]:
field = browser.find_element_by_class_name("news04").text
browser.find_element_by_class_name("news04").click()

dom = BeautifulSoup(browser.page_source, "lxml")

links = [_["href"] for _ in dom.select("#newsContents a") if _.has_attr('href')]

nate_news_detail_crawl(links)

### 사회

In [ ]:
field = browser.find_element_by_class_name("news05").text
browser.find_element_by_class_name("news05").click()

dom = BeautifulSoup(browser.page_source, "lxml")

links = [_["href"] for _ in dom.select("#newsContents a") if _.has_attr('href')]

nate_news_detail_crawl(links)

### 세계

In [ ]:
field = browser.find_element_by_class_name("news06").text
browser.find_element_by_class_name("news06").click()

dom = BeautifulSoup(browser.page_source, "lxml")

links = [_["href"] for _ in dom.select("#newsContents a") if _.has_attr('href')]

nate_news_detail_crawl(links)

### IT/과학

In [ ]:
field = browser.find_element_by_class_name("news07").text[:2]
browser.find_element_by_class_name("news07").click()

dom = BeautifulSoup(browser.page_source, "lxml")

links = [_["href"] for _ in dom.select("#newsContents a") if _.has_attr('href')]

nate_news_detail_crawl(links)

# 번외 : 네이버 뉴스
## 이번엔 beautifulsoup만 써서 많이 본 뉴스 가져오기

In [ ]:
url = "https://news.naver.com/"
html = getDownload(url)
dom = BeautifulSoup(html.content, "lxml")

### 필드 리스트 따로뽑고 10개씩 짤라서 하기

In [ ]:
field_list = dom.select(".category_ranking")[0].text.split(" ")
field_list = list(filter(None, field_list))
len(field_list)

### 랭크가 10개씩 있으니 10개될때마다 1로 리셋하면서 긁어오자

In [ ]:
dom.select(".section_list_ranking a")[-1]["href"].split("=")[5].split("&")[0]

In [ ]:
rank = 1
index = 0
field = field_list[index]
links = dom.select(".section_list_ranking a")

for _ in links:
    
    link = requests.compat.urljoin(url, _["href"])
    html = getDownload(link)
    detail_dom = BeautifulSoup(html.content, "lxml")
    
    # /로 필드이름이 있으면 디렉토리로 인식해서 예외문을 만들었다.
    try:
        file_name = "./news_crawl_project/" + field + "-naver" + _["href"].split("=")[5].split("&")[0]
        f = open(file_name, 'w', encoding="utf-8")
    except FileNotFoundError:
        file_name = "./news_crawl_project/" + field.split("/")[0] + "-naver" + _["href"].split("=")[5].split("&")[0]
        f = open(file_name, 'w', encoding="utf-8")


    for _ in detail_dom.select("#articleBodyContents"):
        f.write(_.text)

    print(file_name, "생성 완료")
#     time.sleep(1)
   
    rank += 1
    
    # rank랑 field 리셋
    if rank > 10 :
        index += 1
        rank = 1
        if index < len(field_list):
            field = field_list[index]
        
        
